### Task 4

In [2]:
import numpy as np
import pandas as pd
import itertools

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint

In [3]:
limphoma_data = np.array([49, 58, 75, 110, 112, 132, 151, 276, 281, 362])
m0 = 200

In [4]:
print("M: {}, p-value: {}".format(*stats.wilcoxon(limphoma_data - m0)))

M: 17.0, p-value: 0.2845026979112075


### Task 5

In [5]:
trees1 = [22,22,15,13,19,19,18,20,21,13,13,15]
trees2 = [17,18,18,15,12,4,14,15,10]

In [6]:
stats.mannwhitneyu(trees1, trees2, alternative='greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.029004992720873729)

### Task 6

In [7]:
challanger_data = pd.read_csv('challenger.txt', sep='\t')

In [8]:
challanger_data.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [9]:
bad_data = challanger_data[challanger_data['Incident'] == 1]['Temperature'].values
good_data = challanger_data[challanger_data['Incident'] == 0]['Temperature'].values

In [10]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [11]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [28]:
np.random.seed(0)

bad_data_boot = list(map(np.mean, get_bootstrap_samples(bad_data, 100)))
good_data_boot = list(map(np.mean, get_bootstrap_samples(good_data, 100)))

In [13]:
delta_mean_scores = list(map(lambda x: x[0] - x[1], zip(bad_data_boot, good_data_boot)))

In [14]:
print("95% confidence interval for the difference between means ", stat_intervals(delta_mean_scores, 0.05))

95% confidence interval for the difference between means  [-8.06457589 -1.45040179]


### Task 7

In [15]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [16]:
def get_random_combinations(n1, n2, max_combinations):
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(list(index))
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [17]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [18]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [29]:
np.random.seed(0)
print("p-value: %f" % permutation_test(good_data_boot, bad_data_boot, max_permutations = 10000))

p-value: 1.000000
